In [2]:
import create_population
import preprocessing
from feature_engineering import FeatureCreator
from feature_engineering_prediction import PredictionFeatureCreator

import prediction


import re
import pandas as pd

import create_population
import html5lib

from train_lightgbm_rank_niti import Trainer_lightgbm_rank_niti
from train_lightgbm_time import Trainer_lightgbm_time
from train_lightgbm_rank_kaiki import Trainer_lightgbm_rank_kaiki


from evaluation_lightgbm_rank_niti import Evaluator_lightgbm_rank_niti
from evaluation_lightgbm_time_kaiki import Evaluator_lightgbm_time_kaiki
from evaluation_lightgbm_rank_kaiki import Evaluator_lightgbm_rank_kaiki
from evaluation_pop import Evaluator_pop

%load_ext autoreload

In [3]:
%autoreload

In [6]:
population = create_population.create(from_="2023-01-01", to_="2023-12-31")


In [6]:
population

,race_id,date,horse_id
0,202306010101,2023-01-05,2020103575
1,202306010101,2023-01-05,2020107073
2,202306010101,2023-01-05,2020102562
3,202306010101,2023-01-05,2020106345
4,202306010101,2023-01-05,2020100039
...,...,...,...
47667,202309050912,2023-12-28,2017104873
47668,202309050912,2023-12-28,2019100653
47669,202309050912,2023-12-28,2017106240
47670,202309050912,2023-12-28,2018103205


In [7]:
#prepro/info 修正

import json
from pathlib import Path

import pandas as pd
import numpy as np
import ast
COMMON_DATA_DIR = Path("..", "..", "common", "data")
RAWDF_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
POPULATION_DIR = Path("..", "data", "00_population")
OUTPUT_DIR = Path("..", "data", "01_preprocessed")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# カテゴリ変数を数値に変換するためのマッピング
with open(MAPPING_DIR / "sex.json", "r") as f:
    sex_mapping = json.load(f)
with open(MAPPING_DIR / "race_type.json", "r") as f:
    race_type_mapping = json.load(f)
with open(MAPPING_DIR / "around.json", "r") as f:
    around_mapping = json.load(f)
with open(MAPPING_DIR / "weather.json", "r") as f:
    weather_mapping = json.load(f)
with open(MAPPING_DIR / "ground_state.json", "r") as f:
    ground_state_mapping = json.load(f)
with open(MAPPING_DIR / "race_class.json", "r") as f:
    race_class_mapping = json.load(f)
with open(MAPPING_DIR / "place.json", "r") as f:
    place_mapping = json.load(f)

population_dir = POPULATION_DIR
populaton_filename = "population.csv"
input_dir= RAWDF_DIR
output_dir = OUTPUT_DIR
input_filename: str = "race_info.csv"
output_filename: str = "race_info.csv"
race_type_mapping = race_type_mapping
weather_mapping = weather_mapping
ground_state_mapping= ground_state_mapping
race_class_mapping = race_class_mapping
place_mapping= place_mapping

In [19]:
population = pd.read_csv(population_dir / populaton_filename, sep="\t")
population_race_ids = population['race_id'].tolist()

# クエリでリストを直接使用
df = pd.read_csv(input_dir / input_filename, sep="\t").query(
    "race_id in @population_race_ids"
)
    
# evalで文字列型の列をリスト型に変換し、一時的な列を作成
df["tmp"] = df["info1"].map(lambda x: eval(x)[0])

# info1 列からコースの長さを取り出して tmp 列を作成
df["tmp2"] = df["info1"].map(lambda x: ast.literal_eval(x))

# tmp 列から距離の部分を抽出
def extract_course_len(info_list):
    for item in info_list:
        match = re.search(r"(\d+)m", item)
        if match:
            return match.group(1)  # マッチした数字部分を返す
    return None  # 該当がなければ None を返す

# コース長を新しい列に追加
df["course_len_1"] = df["tmp2"].apply(extract_course_len)

df["course_len_2"] = df["tmp2"].map(lambda x: x[1]).str.extract(r"(\d+)")
df["combined_course_len"] = df["course_len_1"].fillna(df["course_len_2"])


# ダートor芝or障害
df["race_type"] = df["tmp"].str[0].map(race_type_mapping)
# 右or左or直線
df["around"] = df["tmp"].str[1].map(around_mapping)
df["course_len"] = df["course_len_1"].fillna(df["course_len_2"])
# df["course_len"] = df["tmp"].str.extract(r"(\d+)")
# df["course_len"] = df["tmp"].str.extract(r"(\d+)(?=m)")

df["weather"] = df["info1"].str.extract(r"天候:(\w+)")[0].map(weather_mapping)
df["ground_state"] = (
    df["info1"].str.extract(r"(芝|ダート|障害):(\w+)")[1].map(ground_state_mapping)
)
df["date"] = pd.to_datetime(
    df["info2"].map(lambda x: eval(x)[0]), format="%Y年%m月%d日"
)
regex_race_class = "|".join(race_class_mapping)
df["race_class"] = (
    df["title"]
    .str.extract(rf"({regex_race_class})")
    # タイトルからレース階級情報が取れない場合はinfo2から取得
    .fillna(df["info2"].str.extract(rf"({regex_race_class})"))[0]
    .map(race_class_mapping)
)
df["place"] = df["race_id"].astype(str).str[4:6].astype(int)
df.dropna(subset=["place"], inplace=True)

df = df[df["race_type"] != 2]
population["date"] = pd.to_datetime(population["date"])

df_pop = population.merge(df, on=["race_id", "date"])
df_pop

,race_id,date,horse_id,title,info1,info2,tmp,tmp2,course_len_1,course_len_2,combined_course_len,race_type,around,course_len,weather,ground_state,race_class,place
0,202306010101,2023-01-05,2020103575,3歳未勝利,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:09:55']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '馬齢']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:09:55]",1200,NaN,1200,0,0.0,1200,1,0,1,6
1,202306010101,2023-01-05,2020107073,3歳未勝利,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:09:55']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '馬齢']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:09:55]",1200,NaN,1200,0,0.0,1200,1,0,1,6
2,202306010101,2023-01-05,2020102562,3歳未勝利,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:09:55']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '馬齢']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:09:55]",1200,NaN,1200,0,0.0,1200,1,0,1,6
3,202306010101,2023-01-05,2020106345,3歳未勝利,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:09:55']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '馬齢']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:09:55]",1200,NaN,1200,0,0.0,1200,1,0,1,6
4,202306010101,2023-01-05,2020100039,3歳未勝利,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:09:55']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '馬齢']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:09:55]",1200,NaN,1200,0,0.0,1200,1,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46207,202309050912,2023-12-28,2017104873,カウントダウンS(3勝),"['芝右1200m', '天候:晴', '芝:良', '発走:16:05', '過去のカウン...","['2023年12月28日', '5回阪神9日目', '3歳以上3勝クラス', '混', '...",芝右1200m,"[芝右1200m, 天候:晴, 芝:良, 発走:16:05, 過去のカウントダウンS]",1200,NaN,1200,1,0.0,1200,1,0,4,9
46208,202309050912,2023-12-28,2019100653,カウントダウンS(3勝),"['芝右1200m', '天候:晴', '芝:良', '発走:16:05', '過去のカウン...","['2023年12月28日', '5回阪神9日目', '3歳以上3勝クラス', '混', '...",芝右1200m,"[芝右1200m, 天候:晴, 芝:良, 発走:16:05, 過去のカウントダウンS]",1200,NaN,1200,1,0.0,1200,1,0,4,9
46209,202309050912,2023-12-28,2017106240,カウントダウンS(3勝),"['芝右1200m', '天候:晴', '芝:良', '発走:16:05', '過去のカウン...","['2023年12月28日', '5回阪神9日目', '3歳以上3勝クラス', '混', '...",芝右1200m,"[芝右1200m, 天候:晴, 芝:良, 発走:16:05, 過去のカウントダウンS]",1200,NaN,1200,1,0.0,1200,1,0,4,9
46210,202309050912,2023-12-28,2018103205,カウントダウンS(3勝),"['芝右1200m', '天候:晴', '芝:良', '発走:16:05', '過去のカウン...","['2023年12月28日', '5回阪神9日目', '3歳以上3勝クラス', '混', '...",芝右1200m,"[芝右1200m, 天候:晴, 芝:良, 発走:16:05, 過去のカウントダウンS]",1200,NaN,1200,1,0.0,1200,1,0,4,9


In [20]:
print(df["course_len"].unique())


['1200' '1800' '1600' '2000' '1900' '1400' '3000' '2400' '2200' '1700'
 '1000' '2600' '2100' '1300' '3400' '2500' '1150' '2300' '3200' '1500'
 '3600']


In [15]:
df2 = df[df["race_id"] == 202306050111]
df2

,race_id,title,info1,info2,tmp,tmp2,course_len_1,course_len_2,combined_course_len,race_type,around,course_len,weather,ground_state,date,race_class,place
27085,202306050111,第57回ステイヤーズS(GII),"['芝右内2周3600m', '天候:晴', '芝:良', '発走:15:45', '過去の...","['2023年12月2日', '5回中山1日目', '3歳以上オープン', '国際', '特...",芝右内2周3600m,"[芝右内2周3600m, 天候:晴, 芝:良, 発走:15:45, 過去のステイヤーズS]",3600,NaN,3600,1,0.0,3600,1,0,2023-12-02,7,6


In [28]:
df_pop2 = df[df["course_len"] == "2"]
df_pop2

,race_id,title,info1,info2,tmp,tmp2,course_len_1,course_len_2,combined_course_len,race_type,around,course_len,weather,ground_state,date,race_class,place,date_year,date_month,date_day,month_multiplier,custom_date_value,race_day_count,race_date_day_count
27085,202306050111,第57回ステイヤーズS(GII),"['芝右内2周3600m', '天候:晴', '芝:良', '発走:15:45', '過去のステイヤーズS']","['2023年12月2日', '5回中山1日目', '3歳以上オープン', '国際', '特指', '別定']",芝右内2周3600m,"[芝右内2周3600m, 天候:晴, 芝:良, 発走:15:45, 過去のステイヤーズS]",2,NaN,2,1,0.0,2,1,0,2023-12-02,7,6,2023,12,2,31,738769,11,73876911


In [14]:
import pandas as pd

# サンプルデータ
data = {
    "place": [1,2,1, 2, 1, 3, 4, 1],
    "date": ["2023-04-10","2023-10-10","2023-07-10", "2023-01-15", "2023-12-01", "2023-06-25", "2023-02-14", "2023-08-05"],
}

df = pd.DataFrame(data)

# 日付をdatetime型に変換
df["date"] = pd.to_datetime(df["date"])

# 月を抽出して開催シーズンを判定
def determine_season(month):
    if 6 <= month <= 8:
        return "2" #"夏開催"
    elif month == 12 or 1 <= month <= 2:
        return "4" #"冬開催"
    elif 3 <= month <= 5:
        return "1" #"春開催"
    elif 9 <= month <= 11:
        return "3" #"秋開催"    

df["season"] = df["date"].dt.month.map(determine_season)

# 開催地とシーズンを組み合わせた新しいカテゴリを作成
df["place_season"] = df["place"].astype(str) + "_" + df["season"]

# 結果表示
print(df)


   place       date season place_season
0      1 2023-04-10      1          1_1
1      2 2023-10-10      3          2_3
2      1 2023-07-10      2          1_2
3      2 2023-01-15      4          2_4
4      1 2023-12-01      4          1_4
5      3 2023-06-25      2          3_2
6      4 2023-02-14      4          4_4
7      1 2023-08-05      2          1_2


In [23]:
import pandas as pd

# 行・列・幅の表示制限を解除
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [24]:
population = pd.read_csv(population_dir / populaton_filename, sep="\t")
population_race_ids = population['race_id'].tolist()

# クエリでリストを直接使用
df = pd.read_csv(input_dir / input_filename, sep="\t").query(
    "race_id in @population_race_ids"
)
    
# evalで文字列型の列をリスト型に変換し、一時的な列を作成
df["tmp"] = df["info1"].map(lambda x: eval(x)[0])

# info1 列からコースの長さを取り出して tmp 列を作成
df["tmp2"] = df["info1"].map(lambda x: ast.literal_eval(x))

# tmp 列から距離の部分を抽出
df["course_len_1"] = df["tmp2"].map(lambda x: x[0]).str.extract(r"(\d+)")
df["course_len_2"] = df["tmp2"].map(lambda x: x[1]).str.extract(r"(\d+)")
df["combined_course_len"] = df["course_len_1"].fillna(df["course_len_2"])


# ダートor芝or障害
df["race_type"] = df["tmp"].str[0].map(race_type_mapping)
# 右or左or直線
df["around"] = df["tmp"].str[1].map(around_mapping)
df["course_len"] = df["course_len_1"].fillna(df["course_len_2"])
# df["course_len"] = df["tmp"].str.extract(r"(\d+)")
# df["course_len"] = df["tmp"].str.extract(r"(\d+)(?=m)")

df["weather"] = df["info1"].str.extract(r"天候:(\w+)")[0].map(weather_mapping)
df["ground_state"] = (
    df["info1"].str.extract(r"(芝|ダート|障害):(\w+)")[1].map(ground_state_mapping)
)
df["date"] = pd.to_datetime(
    df["info2"].map(lambda x: eval(x)[0]), format="%Y年%m月%d日"
)
regex_race_class = "|".join(race_class_mapping)
df["race_class"] = (
    df["title"]
    .str.extract(rf"({regex_race_class})")
    # タイトルからレース階級情報が取れない場合はinfo2から取得
    .fillna(df["info2"].str.extract(rf"({regex_race_class})"))[0]
    .map(race_class_mapping)
)
df["place"] = df["race_id"].astype(str).str[4:6].astype(int)
df.dropna(subset=["place"], inplace=True)



# 年、月、日をそれぞれ抽出
df["date_year"] = df["date"].dt.year
df["date_month"] = df["date"].dt.month
df["date_day"] = df["date"].dt.day

# 条件に応じて月の倍率を計算
def get_month_multiplier(month):
    if month in [4, 6, 9, 11]:
        return 30
    elif month == 2:
        return 28
    else:
        return 31

df["month_multiplier"] = df["date_month"].apply(get_month_multiplier)

# 年×365 + 月×倍率 + 日を計算
df["custom_date_value"] = df["date_year"] * 365 + df["month_multiplier"] * df["date_month"] + df["date_day"]

df["race_day_count"] = df['race_id'].astype(str).str[-2:]


df["race_date_day_count"] = df["custom_date_value"].astype(str) + df["race_day_count"]



df["race_day_count"].astype(int)
df["race_date_day_count"].astype(int)

df.head(100)

,race_id,title,info1,info2,tmp,tmp2,course_len_1,course_len_2,combined_course_len,race_type,around,course_len,weather,ground_state,date,race_class,place,date_year,date_month,date_day,month_multiplier,custom_date_value,race_day_count,race_date_day_count
23907,202306010101,3歳未勝利,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:09:55']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '馬齢']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:09:55]",1200,NaN,1200,0,0.0,1200,1,0,2023-01-05,1,6,2023,1,5,31,738431,01,73843101
23908,202306010102,3歳未勝利,"['ダ右1800m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '混', '馬齢']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:10:25]",1800,NaN,1800,0,0.0,1800,1,0,2023-01-05,1,6,2023,1,5,31,738431,02,73843102
23910,202306010104,4歳以上1勝クラス,"['ダ右1800m', '天候:晴', 'ダート:良', '発走:11:25']","['2023年1月5日', '1回中山1日目', '4歳以上1勝クラス', '混', '定量']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:11:25]",1800,NaN,1800,0,0.0,1800,1,0,2023-01-05,2,6,2023,1,5,31,738431,04,73843104
23911,202306010105,3歳未勝利,"['芝右外1600m', '天候:晴', '芝:良', '発走:12:15']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '混', '馬齢']",芝右外1600m,"[芝右外1600m, 天候:晴, 芝:良, 発走:12:15]",1600,NaN,1600,1,0.0,1600,1,0,2023-01-05,1,6,2023,1,5,31,738431,05,73843105
23912,202306010106,3歳1勝クラス,"['芝右2000m', '天候:晴', '芝:良', '発走:12:45']","['2023年1月5日', '1回中山1日目', '3歳1勝クラス', '混', '特指', '馬齢']",芝右2000m,"[芝右2000m, 天候:晴, 芝:良, 発走:12:45]",2000,NaN,2000,1,0.0,2000,1,0,2023-01-05,2,6,2023,1,5,31,738431,06,73843106
23913,202306010107,4歳以上2勝クラス,"['ダ右1800m', '天候:晴', 'ダート:良', '発走:13:15']","['2023年1月5日', '1回中山1日目', '4歳以上2勝クラス', '混', '定量']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:13:15]",1800,NaN,1800,0,0.0,1800,1,0,2023-01-05,3,6,2023,1,5,31,738431,07,73843107
23914,202306010108,4歳以上2勝クラス,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:13:50']","['2023年1月5日', '1回中山1日目', '4歳以上2勝クラス', '定量']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:13:50]",1200,NaN,1200,0,0.0,1200,1,0,2023-01-05,3,6,2023,1,5,31,738431,08,73843108
23915,202306010109,招福ステークス(3勝),"['ダ右1800m', '天候:晴', 'ダート:良', '発走:14:25', '過去の招福ステークス']","['2023年1月5日', '1回中山1日目', '4歳以上3勝クラス', '混', '特指', '定量']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:14:25, 過去の招福ステークス]",1800,NaN,1800,0,0.0,1800,1,0,2023-01-05,4,6,2023,1,5,31,738431,09,73843109
23916,202306010110,ジュニアカップ(L),"['芝右外1600m', '天候:晴', '芝:良', '発走:15:01', '過去のジュニアカップ']","['2023年1月5日', '1回中山1日目', '3歳オープン', '国際', '特指', '別定']",芝右外1600m,"[芝右外1600m, 天候:晴, 芝:良, 発走:15:01, 過去のジュニアカップ]",1600,NaN,1600,1,0.0,1600,1,0,2023-01-05,5,6,2023,1,5,31,738431,10,73843110
23917,202306010111,第72回日刊スポ賞中山金杯(GIII),"['芝右2000m', '天候:晴', '芝:良', '発走:15:35', '過去の日刊スポ賞中山金杯']","['2023年1月5日', '1回中山1日目', '4歳以上オープン', '国際', '特指', 'ハンデ']",芝右2000m,"[芝右2000m, 天候:晴, 芝:良, 発走:15:35, 過去の日刊スポ賞中山金杯]",2000,NaN,2000,1,0.0,2000,1,0,2023-01-05,6,6,2023,1,5,31,738431,11,73843111


In [106]:
population = pd.read_csv(population_dir / populaton_filename, sep="\t")
population_race_ids = population['race_id'].tolist()

# クエリでリストを直接使用
df = pd.read_csv(input_dir / input_filename, sep="\t").query(
    "race_id in @population_race_ids"
)
    
# evalで文字列型の列をリスト型に変換し、一時的な列を作成
df["tmp"] = df["info1"].map(lambda x: eval(x)[0])

# info1 列からコースの長さを取り出して tmp 列を作成
df["tmp2"] = df["info1"].map(lambda x: ast.literal_eval(x))

# tmp 列から距離の部分を抽出
df["course_len_1"] = df["tmp2"].map(lambda x: x[0]).str.extract(r"(\d+)")
df["course_len_2"] = df["tmp2"].map(lambda x: x[1]).str.extract(r"(\d+)")
df["combined_course_len"] = df["course_len_1"].fillna(df["course_len_2"])


# ダートor芝or障害
df["race_type"] = df["tmp"].str[0].map(race_type_mapping)
# 右or左or直線
df["around"] = df["tmp"].str[1].map(around_mapping)
df["course_len"] = df["course_len_1"].fillna(df["course_len_2"])
# df["course_len"] = df["tmp"].str.extract(r"(\d+)")
# df["course_len"] = df["tmp"].str.extract(r"(\d+)(?=m)")

df["weather"] = df["info1"].str.extract(r"天候:(\w+)")[0].map(weather_mapping)
df["ground_state"] = (
    df["info1"].str.extract(r"(芝|ダート|障害):(\w+)")[1].map(ground_state_mapping)
)
df["date"] = pd.to_datetime(
    df["info2"].map(lambda x: eval(x)[0]), format="%Y年%m月%d日"
)
regex_race_class = "|".join(race_class_mapping)
df["race_class"] = (
    df["title"]
    .str.extract(rf"({regex_race_class})")
    # タイトルからレース階級情報が取れない場合はinfo2から取得
    .fillna(df["info2"].str.extract(rf"({regex_race_class})"))[0]
    .map(race_class_mapping)
)
df["place"] = df["race_id"].astype(str).str[4:6].astype(int)
df.dropna(subset=["place"], inplace=True)





# 月を抽出して開催シーズンを判定
def determine_season(month):
    if 6 <= month <= 8:
        return "2" #"夏開催"
    elif month == 12 or 1 <= month <= 2:
        return "4" #"冬開催"
    elif 3 <= month <= 5:
        return "1" #"春開催"
    elif 9 <= month <= 11:
        return "3" #"秋開催"    


df["race_type"] = df["race_type"].astype(str)

df["season"] = df["date"].dt.month.map(determine_season)

# 開催地とシーズンを組み合わせた新しいカテゴリを作成
df["place_season"] = df["place"].astype(str) + df["season"]
#芝/ダートを紐付け、ダ0芝1障2
df["place_season_type"] = df["place_season"] + df["race_type"]



# 開催レースが何回かを表示
df["kaisai_race"] = df['race_id'].astype(str).str[-2:]
#芝/ダートを紐付け、ダ0芝1障2
df["kaisai_race_type"] = df["kaisai_race"] + df["race_type"]

# 何日目かを表示
df["day"] = df['race_id'].astype(str).str[-4:-2]
#芝/ダートを紐付け、ダ0芝1障2
df["day_type"] = df["day"] + df["race_type"]

# 開催回数を表示
df["kaisai_count"] = df['race_id'].astype(str).str[-6:-4]
#芝/ダートを紐付け、ダ0芝1障2
df["kaisai_count_type"] = df["kaisai_count"] + df["race_type"]


# 開催_季節_開催回数を表示
df["place_season_day_type"] = df["place_season"] + df["day_type"]


# condition
df['day_condition'] = np.where(df['day'].astype(int) < 4, 1, 2)
df['day_condition'] = df['day_condition'].astype(str)

# 開催_季節_開催コンディション_レースタイプを表示
df["place_season_condition_type"] = df["place_season"] + df['day_condition'] + df["race_type"]


# コース別の馬場レベルを対応させる
"""
オール野芝 ▶︎最も軽い芝で、速い時計が出やすい
軽い芝 ▶︎時計面では野芝に劣るものの、こちらも速い時計が出やすい
オーバーシード ▶︎ケースバイケース
重い芝 ▶︎馬場が重く、時計もかかりやすい
非常に重い芝 ▶︎JRA10場の中で最も重く、時計もかかりやすい

オール野芝5	 '10111','10211',   小倉(夏)	'6311',中山(秋)	 '9311',   阪神(秋)	 '4111',  '4211',  '4311', '4411', 
新潟	 '7311',   中京(秋)		
軽い芝4	 '5111', '5211',  '5311',  '5411', 
 東京	 '8111',  '8211', '8311',   京都	'4121','4221','4321', '4421',  新潟	'10121','10221','小倉(夏)	 '6321', ' 中山(秋)	'9321', 阪神(秋)	'7321', 中京(秋)
オーバーシード3	'5121','5221','5321','5421',  東京	'8121','8221', '8321',京都	 '9111',  '9211', '9411', 阪神	 '3111', '3211',  '3311', '3411', 
福島	'7111',  '7211',  '7411', 中京	'6111',  '6211',  '6411',  中山	'10311','10411'小倉(冬)
重い芝2	'9121','9221','9421',阪神	'6121','6221','6421',中山	'7121','7221','7421',中京	'3121', '3221','3321', '3421',  福島	'8411', '8421'京都(冬)	 '10321', '10421'小倉(冬)	
非常に重い芝1	 '2111', '2121', '2211', '2221', '2311', '2321', '2411', '2421',  
函館	'1111', '1121', '1211', '1221', '1311', '1321', '1411', '1421',  札幌					

令和版ダートコースの馬場レベル表
軽いダート-1	 '5110', '5120', '5210', '5220', '5310', '5320', '5410', '5420',東京	 '8110', '8120', '8210', '8220', '8310', '8320', '8410', '8420', 京都				
重いダート-2	'1110', '1120', '1210', '1220', '1310', '1320', '1410', '1420', 札幌	'2110', '2120', '2210', '2220', '2310', '2320', '2410', '2420',函館	   '10110', '10120', '10210', '10220', '10310', '10320', '10410', '10420'小倉	'3110', '3120', '3210', '3220', '3310', '3320', '3410', '3420',福島	'4110', '4120', '4210', '4220', '4310', '4320', '4410', '4420',新潟	 '9110', '9120', '9210', '9220', '9310', '9320', '9410', '9420', 阪神
非常に重いダート-3	'7110', '7120', '7210', '7220', '7310', '7320', '7410', '7420', 中京	'6110', '6120', '6210', '6220', '6310', '6320', '6410', '6420', 中山				
障害レースはその他0
"""


conversion_map = {
    '10111': 5, '10211': 5, '6311': 5, '9311': 5, '4111': 5, '4211': 5, '4311': 5, '4411': 5, '7311': 5,
    '5111': 4, '5211': 4, '5311': 4, '5411': 4, '8111': 4, '8211': 4, '8311': 4, '4121': 4, '4221': 4, '4321': 4, '4421': 4,
    '10121': 4, '10221': 4, '6321': 4, '9321': 4, '7321': 4,
    '5121': 3, '5221': 3, '5321': 3, '5421': 3, '8121': 3, '8221': 3, '8321': 3, '9111': 3, '9211': 3, '9411': 3,
    '3111': 3, '3211': 3, '3311': 3, '3411': 3, '7111': 3, '7211': 3, '7411': 3, '6111': 3, '6211': 3, '6411': 3,
    '10311': 3, '10411': 3,
    '9121': 2, '9221': 2, '9421': 2, '6121': 2, '6221': 2, '6421': 2, '7121': 2, '7221': 2, '7421': 2, '3121': 2, 
    '3221': 2, '3321': 2, '3421': 2, '8411': 2, '8421': 2, '10321': 2, '10421': 2,
    '2111': 1, '2121': 1, '2211': 1, '2221': 1, '2311': 1, '2321': 1, '2411': 1, '2421': 1,
    '1111': 1, '1121': 1, '1211': 1, '1221': 1, '1311': 1, '1321': 1, '1411': 1, '1421': 1,
    '5110': -1, '5120': -1, '5210': -1, '5220': -1, '5310': -1, '5320': -1, '5410': -1, '5420': -1,
    '8110': -1, '8120': -1, '8210': -1, '8220': -1, '8310': -1, '8320': -1, '8410': -1, '8420': -1,
    '1110': -2, '1120': -2, '1210': -2, '1220': -2, '1310': -2, '1320': -2, '1410': -2, '1420': -2,
    '2110': -2, '2120': -2, '2210': -2, '2220': -2, '2310': -2, '2320': -2, '2410': -2, '2420': -2,
    '10110': -2, '10120': -2, '10210': -2, '10220': -2, '10310': -2, '10320': -2, '10410': -2, '10420': -2,
    '3110': -2, '3120': -2, '3210': -2, '3220': -2, '3310': -2, '3320': -2, '3410': -2, '3420': -2,
    '4110': -2, '4120': -2, '4210': -2, '4220': -2, '4310': -2, '4320': -2, '4410': -2, '4420': -2,
    '9110': -2, '9120': -2, '9210': -2, '9220': -2, '9310': -2, '9320': -2, '9410': -2, '9420': -2,
    '7110': -3, '7120': -3, '7210': -3, '7220': -3, '7310': -3, '7320': -3, '7410': -3, '7420': -3,
    '6110': -3, '6120': -3, '6210': -3, '6220': -3, '6310': -3, '6320': -3, '6410': -3, '6420': -3
}

df['place_season_condition_type_categori'] = df['place_season_condition_type'].map(conversion_map).fillna(-100).astype(int)
df['place_season_condition_type_categori'] = df['place_season_condition_type_categori'].replace(-100, np.nan)





df["place"] = df["place"].astype(int)
df["course_len"] = df["course_len"].astype(int)

df["race_type"] = df["race_type"].astype(int)

#芝のコース詳細データ
classification_map = {
    9: {  # 阪神
        "内": [1200, 1400, 2000, 2200, 3000, 3200],
        "外": [1400, 1600, 1800, 2400, 2600],
    },
    8: {  # 京都
        "内": [1100, 1200],
        "外": [1800, 2200, 2400, 3000, 3200],
        "内外": [1400, 1600, 2000],
    },
    4: {  # 新潟
        "内": [1200, 1400, 2000, 2200, 2400],
        "外": [1400, 1600, 1800, 2000, 3000, 3200],
        "内外": [],
        "直線": [1000],
    },
    6: {  # 中山
        "内": [1800, 2000, 2500, 3600],
        "外": [1200, 1600, 2200, 2600, 4000],
        "内外": [3200],
    }
}
"""
阪神内
1,200m、1,400m、2,000m
2,200m、3,000m、3,200m

阪神外
1,400m、1,600m、1,800m
2,400m、2,600m

京都内
1,100m(内)、1,200m(内)

京都内・外
1,400m(内・外)、1,600m(内・外)
2,000m(内・外)

京都外
1,800m(外)
2,200m(外)、2,400m(外)
3,000m(外)、3,200m(外)

新潟直線
1,000m

新潟内
1,200m、1,400m
2,000m、2,200m、2,400m

新潟外
1,400m、1,600m
1,800m、2,000m、3,000m、3,200m

中山内外
3,200m（外・内）

中山内
1,800m（内）、2,000m（内）
2,500m（内）、3,600m（内）

中山外
1,200m（外）、1,600m（外）
2,200m（外）、2,600m（外）、4,000m（外）
"""
# 修正された分類ロジック
def classify_place_course(row):
    place = row["place"]
    course_len = row["course_len"]
    race_type = row["race_type"]  # 0: ダート, 1: 芝, 2: 障害
    
    # ダートまたは障害の場合は NaN を返す
    if race_type in [0, 2]:
        return None
    
    place_map = classification_map.get(place, {})
    for category, distances in place_map.items():
        if course_len in distances:
            category_index = list(place_map.keys()).index(category) + 1
            return int(f"{place}{category_index:02d}")  # 整数で返す
    
    # 分類ルールに該当しない場合は競馬場データをそのまま返す
    return place
df["place_course_category"] = df.apply(classify_place_course, axis=1)
#-1を欠損値として扱いintに直す
df["place_course_category"] = df["place_course_category"].fillna(-1).astype(int)


"""
コース別タフ度（芝）
軽い3	新潟	東京	京都外	阪神外
中間2	京都内	小倉	福島	中京
タフ1	阪神内	函館	中山	札幌
"""
# コース別タフ度（芝）_変換ルール
conversion_map = {
    401: 3, 402: 3, 404: 3, 5: 3, 802: 3, 902: 3,
    801: 2, 803: 2, 10: 2, 3: 2, 7: 2,
    901: 1, 2: 1, 6: 1, 601: 1, 602: 1, 1: 1
}
# 新しい列を追加
df["place_course_tough"] = df["place_course_category"].map(conversion_map).fillna(-1).astype(int)

"""
競馬場	直線/m	カーブ	ゴール前
新潟直線	1000	nan	平坦
新潟外	658.7	急	平坦
東京	525.9	複合	緩坂
阪神外	473.6	複合	急坂
中京	412.5	スパ	急坂
京都外	403.7	複合	平坦
京都内・外	345	複合	平坦
新潟内	358.7	急	平坦
阪神内	356.5	複合	急坂
京都内	328.4	複合	平坦
中山内	310	小回	急坂
中山外	310	複合	急坂
中山内外	310	複合	急坂
札幌	266.1	大回	平坦
函館	262.1	小回	平坦
福島	292	小スパ	平坦
小倉	293	小スパ	平坦
"""

# 競馬場カテゴリと対応する直線、カーブ、ゴール前情報
conversion_map = {
    401: {"直線": 1000, "カーブ": None, "ゴール前": "平坦"},
    402: {"直線": 658.7, "カーブ": "急", "ゴール前": "平坦"},
    404: {"直線": 525.9, "カーブ": "複合", "ゴール前": "緩坂"},
    902: {"直線": 473.6, "カーブ": "複合", "ゴール前": "急坂"},
    801: {"直線": 412.5, "カーブ": "スパ", "ゴール前": "急坂"},
    802: {"直線": 403.7, "カーブ": "複合", "ゴール前": "平坦"},
    803: {"直線": 345, "カーブ": "複合", "ゴール前": "平坦"},
    901: {"直線": 358.7, "カーブ": "急", "ゴール前": "平坦"},
    601: {"直線": 310, "カーブ": "小回", "ゴール前": "急坂"},
    602: {"直線": 310, "カーブ": "複合", "ゴール前": "急坂"},
    603: {"直線": 310, "カーブ": "複合", "ゴール前": "急坂"},
    1: {"直線": 266.1, "カーブ": "大回", "ゴール前": "平坦"},
    2: {"直線": 262.1, "カーブ": "小回", "ゴール前": "平坦"},
    3: {"直線": 292, "カーブ": "小スパ", "ゴール前": "平坦"},
    10: {"直線": 293, "カーブ": "小スパ", "ゴール前": "平坦"}
}

# カーブとゴール前を数値に変換する辞書
curve_map = {
    "急": 1,
    "小回": 2,
    "小スパ": 3,
    "スパ": 4,
    "複合": 5,
    None: None  # Noneのまま
}

goal_map = {
    "緩坂": 3,
    "平坦": 2,
    "急坂": 1,
    None: None  # Noneのまま
}

# データフレームに変換情報を適用する関数
def convert_course(row):
    place_code = row["place_course_category"]  # 競馬場の数値コード
    if place_code in conversion_map:
        # 直線、カーブ、ゴール前の情報を取得
        course_info = conversion_map[place_code]
        # カーブとゴール前を数値に変換
        course_info["カーブ"] = curve_map.get(course_info["カーブ"], None)
        course_info["ゴール前"] = goal_map.get(course_info["ゴール前"], None)
        # 列名を変更
        return pd.Series({
            "goal_range": course_info["直線"], 
            "curve": course_info["カーブ"], 
            "goal_slope": course_info["ゴール前"]
        })
    else:
        return pd.Series({"goal_range": None, "curve": None, "goal_slope": None})

# 競馬場カテゴリに基づく変換を追加
df[['goal_range', 'curve', 'goal_slope']] = df.apply(convert_course, axis=1)




#競馬場_季節_芝ダート障害_長さ
df["place_season_type_course_len"] = df["place_season_type"] + df["course_len"].astype(str)

"""
ラップの緩急差が小さいコース
中山芝1600
阪神芝1400
東京芝1400
東京芝1600

緩急のあるラップで脚を溜めやすいコース
京都芝1200
京都芝1400外
中山(秋)芝1200
中山芝2000
阪神芝1200
阪神芝1600外
阪神芝1800外
阪神芝2400外
新潟芝1200、1400内
札幌芝1200、1800、2000
東京芝1800、2000、2400

ロングスパート戦になりやすいコース
東京芝1600、2400
中山芝1800、2000、2200
京都芝1600、1800、2200、2400外
京都芝2000内
阪神芝2000、2200内、2400外
福島芝1800、2000、1700
新潟芝1600、1800、2000外、2200内
小倉芝1800、2000
中京芝1600、2000、2200
札幌芝1500、2000、2600


失速ラップになりやすいコース
中山芝1200
福島芝1200
小倉芝1200
新潟芝1000
函館芝1200、1800、2000
中京芝1200、1400、1600、2000
"""

# マッピングを定義
mapping = {
    # ラップの緩急差が小さいコース → 4
    "6111600": 4, "6211600": 4, "6311600": 4, "6411600": 4,  # 中山(春/夏/秋/冬)芝1600
    "9111400": 4, "9211400": 4, "9311400": 4, "9411400": 4,  # 阪神(春/夏/秋/冬)芝1400
    "5111400": 4, "5211400": 4, "5311400": 4, "5411400": 4,  # 東京(春/夏/秋/冬)芝1400
    "5111600": 4, "5211600": 4, "5311600": 4, "5411600": 4,  # 東京(春/夏/秋/冬)芝1600

    # 緩急のあるラップで脚を溜めやすいコース → 3
    "8111200": 3, "8211200": 3, "8311200": 3, "8411200": 3,  # 京都(春/夏/秋/冬)芝1200
    "8111400": 3, "8211400": 3, "8311400": 3, "8411400": 3,  # 京都(春/夏/秋/冬)芝1400外
    "6311200": 3,  # 中山(秋)芝1200
    "6112000": 3, "6212000": 3, "6312000": 3, "6412000": 3,  # 中山(春/夏/秋/冬)芝2000
    "9111200": 3, "9211200": 3, "9311200": 3, "9411200": 3,  # 阪神(春/夏/秋/冬)芝1200
    "9111600": 3, "9211600": 3, "9311600": 3, "9411600": 3,  # 阪神(春/夏』/秋/冬)芝1600外
    "9111800": 3, "9211800": 3, "9311800": 3, "9411800": 3,  # 阪神(春/夏/秋/冬)芝1800外
    "9112400": 3, "9212400": 3, "9312400": 3, "9412400": 3,  # 阪神(春/夏/秋/冬)芝2400外

    # ロングスパート戦になりやすいコース → 2
    "5111600": 2, "5211600": 2, "5311600": 2, "5411600": 2,  # 東京(春/夏/秋/冬)芝1600
    "5112400": 2, "5212400": 2, "5312400": 2, "5412400": 2,  # 東京(春/夏/秋/冬)芝2400
    "6111800": 2, "6211800": 2, "6311800": 2, "6411800": 2,  # 中山(春/夏/秋/冬)芝1800
    "6112000": 2, "6212000": 2, "6312000": 2, "6412000": 2,  # 中山(春/夏/秋/冬)芝2000
    "6112200": 2, "6212200": 2, "6312200": 2, "6412200": 2,  # 中山(春/夏/秋/冬)芝2200
    "8111600": 2, "8211600": 2, "8311600": 2, "8411600": 2,  # 京都(春/夏/秋/冬)芝1600外
    "8111800": 2, "8211800": 2, "8311800": 2, "8411800": 2,  # 京都(春/夏/秋/冬)芝1800外
    "8112200": 2, "8212200": 2, "8312200": 2, "8412200": 2,  # 京都(春/夏/秋/冬)芝2200外
    "8112400": 2, "8212400": 2, "8312400": 2, "8412400": 2,  # 京都(春/夏/秋/冬)芝2400外

    # 失速ラップになりやすいコース → 1
    "6111200": 1, "6211200": 1, "6411200": 1,  # 中山(春/夏/冬)芝1200
    "3111200": 1, "3211200": 1, "3311200": 1, "3411200": 1,  # 福島(春/夏/秋/冬)芝1200
    "10111200": 1, "11111200": 1, "12111200": 1, "13111200": 1,  # 小倉(春/夏/秋/冬)芝1200
    "4111000": 1, "4211000": 1, "4311000": 1, "4411000": 1,  # 新潟(春/夏/秋/冬)芝1000
    "2111200": 1, "2211200": 1, "2311200": 1, "2411200": 1,  # 函館(春/夏/秋/冬)芝1200
    "7111200": 1, "7211200": 1, "7311200": 1, "7411200": 1,  # 中京(春/夏/秋/冬)芝1200
}

# `place_season_type_course_len` をマッピングに基づいて変換
df["lap_type"] = df["place_season_type_course_len"].map(mapping)
# マッピングできなかったものを NaN にする
df['lap_type'] = df['lap_type'].fillna(-1)
df['lap_type'] = df['lap_type'].astype(int)




# 各列をint型に変換

df["race_class"] = df["race_class"].astype(int)
df["ground_state"] = df["ground_state"].astype(int)
df["around"] = df["around"].fillna(3).astype(int)
df["weather"] = df["weather"].astype(int)



df[['goal_range', 'curve', 'goal_slope']] = df[['goal_range', 'curve', 'goal_slope']].fillna(-1).astype(int)
df["place_season_type_course_len"] = df["place_season_type_course_len"].fillna(-1).astype(int)
df['lap_type'] = df['lap_type'].fillna(-1).astype(int)


df["course_len"] = df["course_len"].astype(int)
df["season"] = df["season"].astype(int)
df["place_season"] = df["place_season"].astype(int)
df["place_season_type"] = df["place_season_type"].astype(int)
df["kaisai_race"] = df["kaisai_race"].astype(int)
df["kaisai_race_type"] = df["kaisai_race_type"].astype(int)
df["day"] = df["day"].astype(int)
df["day_type"] = df["day_type"].astype(int)
df["kaisai_count"] = df["kaisai_count"].astype(int)
df["kaisai_count_type"] = df["kaisai_count_type"].astype(int)
df["place_season_day_type"] = df["place_season_day_type"].astype(int)
df["day_condition"] = df["day_condition"].astype(int)
df["place_season_condition_type"] = df["place_season_condition_type"].astype(int)

df['goal_range'] = df['goal_range'].fillna(-1).astype(int)
df['curve'] = df['curve'].fillna(-1).astype(int)
df['goal_slope'] = df['goal_slope'].fillna(-1).astype(int)





df.head(100)# 列の種類を取得


,race_id,title,info1,info2,tmp,tmp2,course_len_1,course_len_2,combined_course_len,race_type,around,course_len,weather,ground_state,date,race_class,place,season,place_season,place_season_type,kaisai_race,kaisai_race_type,day,day_type,kaisai_count,kaisai_count_type,place_season_day_type,day_condition,place_season_condition_type,place_season_condition_type_categori,place_course_category,place_course_tough,goal_range,curve,goal_slope,place_season_type_course_len,lap_type
23907,202306010101,3歳未勝利,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:09:55']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '馬齢']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:09:55]",1200,NaN,1200,0,0,1200,1,0,2023-01-05,1,6,4,64,640,1,10,1,10,1,10,64010,1,6410,-3.0,-1,-1,-1,-1,-1,6401200,-1
23908,202306010102,3歳未勝利,"['ダ右1800m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '混', '馬齢']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:10:25]",1800,NaN,1800,0,0,1800,1,0,2023-01-05,1,6,4,64,640,2,20,1,10,1,10,64010,1,6410,-3.0,-1,-1,-1,-1,-1,6401800,-1
23909,202306010103,3歳新馬,"['ダ右1800m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年1月5日', '1回中山1日目', '3歳新馬', '混', '馬齢']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:10:55]",1800,NaN,1800,0,0,1800,1,0,2023-01-05,0,6,4,64,640,3,30,1,10,1,10,64010,1,6410,-3.0,-1,-1,-1,-1,-1,6401800,-1
23910,202306010104,4歳以上1勝クラス,"['ダ右1800m', '天候:晴', 'ダート:良', '発走:11:25']","['2023年1月5日', '1回中山1日目', '4歳以上1勝クラス', '混', '定量']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:11:25]",1800,NaN,1800,0,0,1800,1,0,2023-01-05,2,6,4,64,640,4,40,1,10,1,10,64010,1,6410,-3.0,-1,-1,-1,-1,-1,6401800,-1
23911,202306010105,3歳未勝利,"['芝右外1600m', '天候:晴', '芝:良', '発走:12:15']","['2023年1月5日', '1回中山1日目', '3歳未勝利', '混', '馬齢']",芝右外1600m,"[芝右外1600m, 天候:晴, 芝:良, 発走:12:15]",1600,NaN,1600,1,0,1600,1,0,2023-01-05,1,6,4,64,641,5,51,1,11,1,11,64011,1,6411,3.0,602,1,310,5,1,6411600,4
23912,202306010106,3歳1勝クラス,"['芝右2000m', '天候:晴', '芝:良', '発走:12:45']","['2023年1月5日', '1回中山1日目', '3歳1勝クラス', '混', '特指', '馬齢']",芝右2000m,"[芝右2000m, 天候:晴, 芝:良, 発走:12:45]",2000,NaN,2000,1,0,2000,1,0,2023-01-05,2,6,4,64,641,6,61,1,11,1,11,64011,1,6411,3.0,601,1,310,2,1,6412000,2
23913,202306010107,4歳以上2勝クラス,"['ダ右1800m', '天候:晴', 'ダート:良', '発走:13:15']","['2023年1月5日', '1回中山1日目', '4歳以上2勝クラス', '混', '定量']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:13:15]",1800,NaN,1800,0,0,1800,1,0,2023-01-05,3,6,4,64,640,7,70,1,10,1,10,64010,1,6410,-3.0,-1,-1,-1,-1,-1,6401800,-1
23914,202306010108,4歳以上2勝クラス,"['ダ右1200m', '天候:晴', 'ダート:良', '発走:13:50']","['2023年1月5日', '1回中山1日目', '4歳以上2勝クラス', '定量']",ダ右1200m,"[ダ右1200m, 天候:晴, ダート:良, 発走:13:50]",1200,NaN,1200,0,0,1200,1,0,2023-01-05,3,6,4,64,640,8,80,1,10,1,10,64010,1,6410,-3.0,-1,-1,-1,-1,-1,6401200,-1
23915,202306010109,招福ステークス(3勝),"['ダ右1800m', '天候:晴', 'ダート:良', '発走:14:25', '過去の招福ステークス']","['2023年1月5日', '1回中山1日目', '4歳以上3勝クラス', '混', '特指', '定量']",ダ右1800m,"[ダ右1800m, 天候:晴, ダート:良, 発走:14:25, 過去の招福ステークス]",1800,NaN,1800,0,0,1800,1,0,2023-01-05,4,6,4,64,640,9,90,1,10,1,10,64010,1,6410,-3.0,-1,-1,-1,-1,-1,6401800,-1
23916,202306010110,ジュニアカップ(L),"['芝右外1600m', '天候:晴', '芝:良', '発走:15:01', '過去のジュニアカップ']","['2023年1月5日', '1回中山1日目', '3歳オープン', '国際', '特指', '別定']",芝右外1600m,"[芝右外1600m, 天候:晴, 芝:良, 発走:15:01, 過去のジュニアカップ]",1600,NaN,1600,1,0,1600,1,0,2023-01-05,5,6,4,64,641,10,101,1,11,1,11,64011,1,6411,3.0,602,1,310,-1,-1,6411600,4


In [100]:
unique_goal_range = sorted(df['goal_range'].unique())
unique_curve = sorted(df['curve'].unique())
unique_goal_slope = sorted(df['goal_slope'].unique())

print(unique_goal_slope )



[np.int64(-1), np.int64(1), np.int64(2), np.int64(3)]


In [89]:
unique_types = sorted(df["place_course_tough"].unique())
print(unique_types)

[np.int64(-1), np.int64(1), np.int64(2), np.int64(3)]


In [85]:

unique_types = sorted(df["place_course_category"].unique())
print(unique_types)

[np.int64(-1), np.int64(1), np.int64(2), np.int64(3), np.int64(5), np.int64(6), np.int64(7), np.int64(10), np.int64(401), np.int64(402), np.int64(404), np.int64(601), np.int64(602), np.int64(801), np.int64(802), np.int64(803), np.int64(901), np.int64(902)]


In [79]:
print(df["course_len"].unique())  # データ内の距離を確認
print(classification_map)         # 設定した距離を確認


['1200' '1800' '1600' '2000' '1900' '1400' '3000' '2400' '3200' '2200'
 '2880' '1700' '1000' '2860' '2600' '2100' '3390' '1300' '3400' '2500'
 '2970' '3900' '3350' '3110' '3140' '2750' '1150' '4250' '2300' '3380'
 '2770' '2890' '3930' '3290' '2910' '3300' '3330' '1500' '3250' '2850'
 '3210' '3170' '3570' '2' '4100']
{9: {'内': [1200, 1400, 2000, 2200, 3000, 3200], '外': [1400, 1600, 1800, 2400, 2600]}, 8: {'内': [1100, 1200], '外': [1800, 2200, 2400, 3000, 3200], '内外': [1400, 1600, 2000]}, 4: {'内': [1200, 1400, 2000, 2200, 2400], '外': [1400, 1600, 1800, 2000, 3000, 3200], '内外': [], '直線': [1000]}, 6: {'内': [1800, 2000, 2500, 3600], '外': [1200, 1600, 2200, 2600, 4000], '内外': [3200]}}


In [87]:
# 列の種類を取得し、昇順でソート
unique_types = sorted(df['place_season_condition_type_categori'].unique())

# 結果表示
print(unique_types)

[np.float64(-3.0), np.float64(3.0), np.float64(nan), np.float64(-2.0), np.float64(-1.0), np.float64(1.0), np.float64(2.0), np.float64(4.0), np.float64(5.0)]


In [58]:
# 列の種類を取得し、昇順でソート
unique_types = sorted(df["place_season_condition_type"].unique())

# 結果表示
print(unique_types)


['10210', '10211', '10212', '10220', '10221', '10222', '10320', '10321', '10322', '10410', '10411', '10412', '10420', '10421', '10422', '1210', '1211', '1220', '1221', '1320', '1321', '2210', '2211', '2220', '2221', '3110', '3111', '3112', '3120', '3121', '3122', '3210', '3211', '3212', '3220', '3221', '3222', '3310', '3311', '3312', '3320', '3321', '3322', '4110', '4111', '4112', '4120', '4121', '4122', '4210', '4211', '4212', '4220', '4221', '4222', '4310', '4311', '4312', '4320', '4321', '4322', '5110', '5111', '5120', '5121', '5122', '5210', '5211', '5212', '5220', '5221', '5222', '5310', '5311', '5312', '5320', '5321', '5322', '5410', '5411', '5420', '5421', '6110', '6111', '6112', '6120', '6121', '6122', '6310', '6311', '6312', '6320', '6321', '6322', '6410', '6411', '6412', '6420', '6421', '6422', '7110', '7111', '7120', '7121', '7210', '7211', '7212', '7220', '7221', '7222', '7410', '7411', '7420', '7421', '8110', '8111', '8120', '8121', '8122', '8310', '8311', '8312', '8320', 

In [59]:

# 種類とその出現回数を取得
type_counts = df["place_season_condition_type"].value_counts()

# 結果表示
print(type_counts)


place_season_condition_type
7420     85
9120     79
6420     74
5321     71
8321     70
6120     68
8320     61
5320     60
7421     59
9420     58
5120     56
6410     55
6421     55
8121     54
10421    53
8120     52
5121     51
9121     50
6121     48
9421     47
2221     43
2211     42
1221     42
1211     42
9410     40
7410     40
4211     40
6411     38
10411    38
6321     37
5311     36
8311     36
9321     36
5420     34
5310     34
7221     34
9320     34
3221     34
8310     33
7411     32
6320     32
10420    32
1220     30
5220     30
2210     30
1210     30
4321     30
9411     30
2220     29
4221     29
9221     29
6110     28
5221     28
9220     28
4210     28
4120     27
4121     27
5421     26
10410    24
4320     24
7220     23
3220     22
10211    22
7120     21
7110     21
10221    21
9110     20
5110     20
3211     20
5410     20
9210     19
3311     19
7211     18
4220     18
5210     18
6111     18
8110     18
8111     18
6310     18
3111     18
9310     17


In [63]:
# 末尾が '1' のデータをフィルタリング
filtered_df = df[df["place_season_condition_type"].astype(str).str.endswith('1')]

# 末尾が '1' のデータの種類を表示
unique_filtered_types = sorted(filtered_df["place_season_condition_type"].unique())

# 結果表示
print(unique_filtered_types)


['10211', '10221', '10321', '10411', '10421', '1211', '1221', '1321', '2211', '2221', '3111', '3121', '3211', '3221', '3311', '3321', '4111', '4121', '4211', '4221', '4311', '4321', '5111', '5121', '5211', '5221', '5311', '5321', '5411', '5421', '6111', '6121', '6311', '6321', '6411', '6421', '7111', '7121', '7211', '7221', '7411', '7421', '8111', '8121', '8311', '8321', '9111', '9121', '9211', '9221', '9311', '9321', '9411', '9421']


In [70]:
# 条件に基づく変換マッピング
conversion_map = {
    '10111': 5, '10211': 5, '6311': 5, '9311': 5, '4111': 5, '4211': 5, '4311': 5, '4411': 5, '7311': 5,
    '5111': 4, '5211': 4, '5311': 4, '5411': 4, '8111': 4, '8211': 4, '8311': 4, '4121': 4, '4221': 4, '4321': 4, '4421': 4,
    '10121': 4, '10221': 4, '6321': 4, '9321': 4, '7321': 4,
    '5121': 3, '5221': 3, '5321': 3, '5421': 3, '8121': 3, '8221': 3, '8321': 3, '9111': 3, '9211': 3, '9411': 3,
    '3111': 3, '3211': 3, '3311': 3, '3411': 3, '7111': 3, '7211': 3, '7411': 3, '6111': 3, '6211': 3, '6411': 3,
    '10311': 3, '10411': 3,
    '9121': 2, '9221': 2, '9421': 2, '6121': 2, '6221': 2, '6421': 2, '7121': 2, '7221': 2, '7421': 2, '3121': 2, 
    '3221': 2, '3321': 2, '3421': 2, '8411': 2, '8421': 2, '10321': 2, '10421': 2,
    '2111': 1, '2121': 1, '2211': 1, '2221': 1, '2311': 1, '2321': 1, '2411': 1, '2421': 1,
    '1111': 1, '1121': 1, '1211': 1, '1221': 1, '1311': 1, '1321': 1, '1411': 1, '1421': 1,
    '5110': -1, '5120': -1, '5210': -1, '5220': -1, '5310': -1, '5320': -1, '5410': -1, '5420': -1,
    '8110': -1, '8120': -1, '8210': -1, '8220': -1, '8310': -1, '8320': -1, '8410': -1, '8420': -1,
    '1110': -2, '1120': -2, '1210': -2, '1220': -2, '1310': -2, '1320': -2, '1410': -2, '1420': -2,
    '2110': -2, '2120': -2, '2210': -2, '2220': -2, '2310': -2, '2320': -2, '2410': -2, '2420': -2,
    '10110': -2, '10120': -2, '10210': -2, '10220': -2, '10310': -2, '10320': -2, '10410': -2, '10420': -2,
    '3110': -2, '3120': -2, '3210': -2, '3220': -2, '3310': -2, '3320': -2, '3410': -2, '3420': -2,
    '4110': -2, '4120': -2, '4210': -2, '4220': -2, '4310': -2, '4320': -2, '4410': -2, '4420': -2,
    '9110': -2, '9120': -2, '9210': -2, '9220': -2, '9310': -2, '9320': -2, '9410': -2, '9420': -2,
    '7110': -3, '7120': -3, '7210': -3, '7220': -3, '7310': -3, '7320': -3, '7410': -3, '7420': -3,
    '6110': -3, '6120': -3, '6210': -3, '6220': -3, '6310': -3, '6320': -3, '6410': -3, '6420': -3
}

# 変換処理
df['place_season_condition_type_categori'] = df['place_season_condition_type'].map(conversion_map).fillna(0).astype(int)

# 結果表示
print(df[['place_season_condition_type','place_season_condition_type_categori']])






      place_season_condition_type  place_season_condition_type_categori
23907                        6410                                    -3
23908                        6410                                    -3
23909                        6410                                    -3
23910                        6410                                    -3
23911                        6411                                     3
23912                        6411                                     3
23913                        6410                                    -3
23914                        6410                                    -3
23915                        6410                                    -3
23916                        6411                                     3
23917                        6411                                     3
23918                        6411                                     3
23919                        7410                               

In [ ]:
['1111', '1121', '1211', '1221', '1311', '1321', '1411', '1421',  
 '2111', '2121', '2211', '2221', '2311', '2321', '2411', '2421',  
 '3111', '3121', '3211', '3221', '3311', '3321', '3411', '3421',  
 '4111', '4121', '4211', '4221', '4311', '4321', '4411', '4421',  
 '5111', '5121', '5211', '5221', '5311', '5321', '5411', '5421',  
 '6111', '6121', '6211', '6221', '6311', '6321', '6411', '6421',  
 '7111', '7121', '7211', '7221', '7311', '7321', '7411', '7421',  
 '8111', '8121', '8211', '8221', '8311', '8321', '8411', '8421',  
 '9111', '9121', '9211', '9221', '9311', '9321', '9411', '9421',  
 '10111', '10121', '10211', '10221', '10311', '10321', '10411', '10421']

 [
    '1110', '1120', '1210', '1220', '1310', '1320', '1410', '1420',
    '2110', '2120', '2210', '2220', '2310', '2320', '2410', '2420',
    '3110', '3120', '3210', '3220', '3310', '3320', '3410', '3420',
    '4110', '4120', '4210', '4220', '4310', '4320', '4410', '4420',
    '5110', '5120', '5210', '5220', '5310', '5320', '5410', '5420',
    '6110', '6120', '6210', '6220', '6310', '6320', '6410', '6420',
    '7110', '7120', '7210', '7220', '7310', '7320', '7410', '7420',
    '8110', '8120', '8210', '8220', '8310', '8320', '8410', '8420',
    '9110', '9120', '9210', '9220', '9310', '9320', '9410', '9420',
    '10110', '10120', '10210', '10220', '10310', '10320', '10410', '10420'
]


In [9]:
#prepro/horse 修正

import json
from pathlib import Path

import pandas as pd
import numpy as np
import ast
COMMON_DATA_DIR = Path("..", "..", "common", "data")
RAWDF_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
POPULATION_DIR = Path("..", "data", "00_population")
OUTPUT_DIR = Path("..", "data", "01_preprocessed")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# カテゴリ変数を数値に変換するためのマッピング
with open(MAPPING_DIR / "sex.json", "r") as f:
    sex_mapping = json.load(f)
with open(MAPPING_DIR / "race_type.json", "r") as f:
    race_type_mapping = json.load(f)
with open(MAPPING_DIR / "around.json", "r") as f:
    around_mapping = json.load(f)
with open(MAPPING_DIR / "weather.json", "r") as f:
    weather_mapping = json.load(f)
with open(MAPPING_DIR / "ground_state.json", "r") as f:
    ground_state_mapping = json.load(f)
with open(MAPPING_DIR / "race_class.json", "r") as f:
    race_class_mapping = json.load(f)
with open(MAPPING_DIR / "place.json", "r") as f:
    place_mapping = json.load(f)

population_dir = POPULATION_DIR
populaton_filename = "population.csv"
input_dir= RAWDF_DIR
output_dir = OUTPUT_DIR
input_filename = "horse_results.csv"
output_filename = "horse_results.csv"
race_type_mapping = race_type_mapping
weather_mapping = weather_mapping
ground_state_mapping= ground_state_mapping
race_class_mapping = race_class_mapping
place_mapping= place_mapping

In [58]:
population = pd.read_csv(population_dir / populaton_filename, sep="\t")
# df = pd.read_csv(input_dir / input_filename, sep="\t").query(
#     "horse_id in @population['horse_id']"
# )
population_horse_ids = population['horse_id'].tolist()

# クエリでリストを直接使用
df = pd.read_csv(input_dir / input_filename, sep="\t").query(
    "horse_id in @population_horse_ids"
)


df["rank"] = pd.to_numeric(df["着順"], errors="coerce")
df.dropna(subset=["rank"], inplace=True)
df["date"] = pd.to_datetime(df["日付"])
df["weather"] = df["天気"].map(weather_mapping)
df["race_type"] = df["距離"].str[0].map(race_type_mapping)
df["course_len"] = df["距離"].str.extract(r"(\d+)").astype(int)
df["ground_state"] = df["馬場"].map(ground_state_mapping)
df["rank_diff"] = df["着差"].map(lambda x: 0 if x < 0 else x)
df["prize"] = df["賞金"].fillna(0)
df["umaban"] = df["馬番"].astype(int)

df["wakuban"] = df["枠番"].fillna(0).astype(int)


df["nobori"] = df["上り"].astype(float)

regex_race_class = "|".join(race_class_mapping)
df["race_class"] = (
    df["レース名"].str.extract(rf"({regex_race_class})")[0].map(race_class_mapping)
)
df["time"] = pd.to_datetime(df["タイム"], format="%M:%S.%f", errors="coerce")
df["time"] = (
    df["time"].dt.minute * 60
    + df["time"].dt.second
    + df["time"].dt.microsecond / 1000000
)
df["win"] = (df["rank"] == 1).astype(int)
df["rentai"] = (df["rank"] <= 2).astype(int)
df["show"] = (df["rank"] <= 3).astype(int)
df["place"] = df["開催"].str.extract(r"(\D+)")[0].map(place_mapping)
df.dropna(subset=["place"], inplace=True)    
df.rename(columns={"頭数": "n_horses"}, inplace=True)

pase_cols = df['ペース'].str.split('-', expand=True)
pase_cols.columns = [f'pase_{i+1}' for i in range(pase_cols.shape[1])]

# データ型変換と型確認
pase_cols = pase_cols.apply(pd.to_numeric, errors='coerce').astype('float64')

# 元のデータフレームと結合
df = pd.concat([df, pase_cols], axis=1)

# NoneをNaNに置き換え
# df = df.where(pd.notnull(df), np.nan)
df["place"] = df["place"].astype(int)
df["race_class"] = df["race_class"].astype(int)
df["ground_state"] = df["ground_state"].astype(int)
df["weather"] = df["weather"].astype(int)   
# コーナー通過順を分割して列を作成
corner_cols = df['通過'].str.split('-', expand=True)
corner_cols.columns = [f'corner_{i+1}' for i in range(corner_cols.shape[1])]
# オブジェクト型のデータを整数型に変換する
corner_cols = corner_cols.apply(pd.to_numeric, errors='coerce').astype('Int64')  
# nullable int型を指定
# 元のデータフレームと結合
df = pd.concat([df, corner_cols], axis=1)

# NoneをNaNに置き換え
df = df.where(pd.notnull(df), np.nan)

# rank / n_horses の特徴量を作成（欠損値を含む行はNaNに設定）
df["rank_per_horse"] = df["rank"].where(df["rank"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)

# corner_1 / n_horses の特徴量を作成（欠損値を含む行はNaNに設定）
df["corner_1_per_horse"] = df["corner_1"].where(df["corner_1"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)

df["corner_2_per_horse"] = df["corner_2"].where(df["corner_2"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)
df["corner_3_per_horse"] = df["corner_3"].where(df["corner_3"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)
df["corner_4_per_horse"] = df["corner_4"].where(df["corner_4"].notna(), np.nan) / df["n_horses"].where(df["n_horses"].notna(), np.nan)
df["time_courselen"] = df["time"] / df["course_len"]
df = df.where(pd.notnull(df), np.nan)
# 使用する列を選択
df = df[
    [
        "horse_id",
        "rank_per_horse",
        "date",
        "rank",
        "prize",
        "rank_diff",
        "umaban",
        "wakuban",
        "weather",
        "race_type",
        "course_len",
        "ground_state",
        "race_class",
        "n_horses",
        "time",
        "time_courselen",
        "nobori",
        "win",
        "rentai",
        "show",
        "place",
        "corner_1_per_horse",
        "corner_2_per_horse",
        "corner_3_per_horse",
        "corner_4_per_horse",
    ] + list(corner_cols.columns) + list(pase_cols.columns)
]

In [14]:
df

,horse_id,rank_per_horse,date,rank,prize,rank_diff,umaban,wakuban,weather,race_type,...,corner_1_per_horse,corner_2_per_horse,corner_3_per_horse,corner_4_per_horse,corner_1,corner_2,corner_3,corner_4,pase_1,pase_2
894,2013105125,1.000000,2023-04-22,14.0,0.0,13.1,6,4,1,2,...,1.0,1.0,1.0,1.0,14,14,14,14,104.1,38.3
895,2013105125,0.785714,2022-04-23,11.0,0.0,5.9,2,2,2,2,...,0.857143,0.857143,0.785714,0.857143,12,12,11,12,105.1,38.6
896,2013105125,0.818182,2021-10-23,9.0,0.0,2.5,3,3,2,2,...,0.818182,0.818182,1.0,0.818182,9,9,11,9,105.0,37.9
897,2013105125,0.909091,2021-07-10,10.0,0.0,3.3,5,5,3,2,...,0.818182,0.818182,0.909091,0.909091,9,9,10,10,106.0,38.1
898,2013105125,0.923077,2021-04-24,12.0,0.0,3.8,12,8,2,2,...,0.923077,0.846154,0.923077,0.846154,12,11,12,11,105.0,38.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077785,2021102427,0.571429,2024-04-06,8.0,0.0,2.3,12,7,2,0,...,0.571429,0.642857,0.428571,0.5,8,9,6,7,39.1,38.0
1077786,2021102427,0.928571,2023-12-28,13.0,0.0,2.5,7,5,1,1,...,0.642857,0.642857,<NA>,<NA>,9,9,<NA>,<NA>,36.0,34.1
1077795,2021103807,1.000000,2024-02-24,11.0,0.0,3.4,1,1,2,0,...,0.090909,0.090909,0.090909,0.181818,1,1,1,2,36.8,39.7
1077796,2021103807,0.785714,2024-01-13,11.0,0.0,2.1,2,2,1,0,...,1.0,0.857143,0.642857,0.571429,14,12,9,8,35.8,39.8


In [59]:
baselog = (
    population.merge
    (
        df,
        on="horse_id",
        suffixes=("", "_horse"),
    )
    .query("date_horse < date")
    .sort_values("date_horse", ascending=False)
)
baselog

,race_id,date,horse_id,rank_per_horse,date_horse,rank,prize,rank_diff,umaban,wakuban,...,corner_1_per_horse,corner_2_per_horse,corner_3_per_horse,corner_4_per_horse,corner_1,corner_2,corner_3,corner_4,pase_1,pase_2
681957,202309050903,2023-12-28,2021105018,0.933333,2023-12-23,14.0,0.0,3.3,8,5,...,0.666667,0.8,<NA>,<NA>,10,12,<NA>,<NA>,35.6,38.0
681940,202309050903,2023-12-28,2021104215,1.000000,2023-12-23,14.0,0.0,2.0,9,6,...,1.0,1.0,<NA>,<NA>,14,14,<NA>,<NA>,35.4,34.4
681899,202309050903,2023-12-28,2021105731,0.857143,2023-12-23,12.0,0.0,1.6,12,7,...,0.428571,0.571429,<NA>,<NA>,6,8,<NA>,<NA>,35.4,34.4
681963,202309050903,2023-12-28,2021102811,1.000000,2023-12-23,17.0,0.0,3.3,10,5,...,0.764706,0.882353,<NA>,<NA>,13,15,<NA>,<NA>,34.8,34.8
674916,202309050712,2023-12-23,2020103735,0.937500,2023-12-17,15.0,0.0,2.1,6,3,...,1.0,1.0,<NA>,<NA>,16,16,<NA>,<NA>,34.6,38.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322860,202309030109,2023-06-03,2011106610,0.375000,2014-01-19,6.0,0.0,0.7,14,7,...,0.125,0.0625,0.125,<NA>,2,1,2,<NA>,36.0,36.7
580180,202308020909,2023-10-29,2011106610,0.375000,2014-01-19,6.0,0.0,0.7,14,7,...,0.125,0.0625,0.125,<NA>,2,1,2,<NA>,36.0,36.7
393604,202307030412,2023-07-09,2011106610,0.375000,2014-01-19,6.0,0.0,0.7,14,7,...,0.125,0.0625,0.125,<NA>,2,1,2,<NA>,36.0,36.7
212844,202309020607,2023-04-09,2011106610,0.375000,2014-01-19,6.0,0.0,0.7,14,7,...,0.125,0.0625,0.125,<NA>,2,1,2,<NA>,36.0,36.7


In [22]:
population

,race_id,date,horse_id
0,202306010101,2023-01-05,2020103575
1,202306010101,2023-01-05,2020107073
2,202306010101,2023-01-05,2020102562
3,202306010101,2023-01-05,2020106345
4,202306010101,2023-01-05,2020100039
...,...,...,...
47667,202309050912,2023-12-28,2017104873
47668,202309050912,2023-12-28,2019100653
47669,202309050912,2023-12-28,2017106240
47670,202309050912,2023-12-28,2018103205


In [66]:
grouped_df = baselog.groupby(["race_id", "horse_id"])
merged_df = population.copy()

df2 = (
    grouped_df.head(3)
    .groupby(["race_id", "horse_id"])[["rank"]]
    .mean()
    .add_suffix(f"_3races")
)
merged_df = merged_df.merge(df2, on=["race_id", "horse_id"], how="left")
merged_df

merged_df["rank_3races"].isnull().sum()

np.int64(89)